In [ ]:
import pandas as pd
import numpy as np
import operator
import math

In [ ]:
u_data = pd.read_csv("u1.base",delimiter='\t',header = None)
u_data.columns = ("user id","item id","rating","timestamp")
u_data

u1_test = pd.read_csv("u1.test",delimiter="\t",header = None)
u1_test.columns = ("user id","item id","rating","timestamp")


In [ ]:
item_data = pd.read_csv("u.item",delimiter = "|",header = None,encoding='latin-1')
item_data.columns = ("movie id", "movie title" , "release date" , "video release date ",
                    "IMDb URL" ,"unknown" , "Action" , "Adventure",  
                     "Animation" ,"Children\'s" , "Comedy" , "Crime" ,
                     "Documentary" ,"Drama" , "Fantasy" ,"Film-Noir" ,
                     "Horror" , "Musical" ,"Mystery" , "Romance" ,
                     "Sci-Fi" ,"Thriller" ,"War" , "Western")


In [ ]:
#del u_data['timestamp']
ratings_df = u_data.pivot_table(values='rating',index='user id',columns = 'item id')
ratings_df.fillna(value=0,inplace=True,axis=1)

In [ ]:
ratings_df

In [ ]:

total_users = 943
total_items = 1682
total_ratings = 100000

In [ ]:
genre_data = pd.read_csv("u.genre",delimiter='|',header = None)
genre_data.columns = ("genre","genre_id")


In [ ]:
user_data = pd.read_csv("u.user",delimiter="|",header=None)
user_data.columns = ("user id","age","gender","occupation","zip code")
len(pd.unique(user_data['zip code']))
for index,eachrow in user_data.iterrows():
    if eachrow['age'] >= 7 and eachrow['age'] <=15:
        user_data.set_value(index,'age',1)
    elif eachrow['age'] >= 16 and eachrow['age'] <=25:
        user_data.set_value(index,'age',2)
    elif eachrow['age'] >= 26 and eachrow['age'] <=31:
        user_data.set_value(index,'age',3)
    elif eachrow['age'] >= 32 and eachrow['age'] <= 43:
        user_data.set_value(index,'age',4)
    else:
        user_data.set_value(index,'age',5)
user_data['gender'].replace(['F','M'],[1,2],inplace=True)
#user_data['occupation'] = user_data.occupation.astype('category').cat.rename_categories(range(1,user_data.occupation.nunique()+1))
merged_set = u_data.merge(user_data, left_on='user id', right_on='user id', how='inner')


In [ ]:
def MAD(rating):
    u1_test.head()
    i = 0
    sum_rij = 0
    sum_predicted_val = 0
    for index,everyrow in u1_test.iterrows():
        tij = everyrow['rating']
        pij = rating[i]
        i = i +1
        if math.isnan(tij) or tij==0:
            rij = 0
        else:

            rij = 1
        sum_rij += rij
        error = abs(pij - tij)
        temp = rij * error
        sum_predicted_val += temp
    mad = sum_predicted_val/sum_rij
    return (mad)
        

In [ ]:
def myratings(current_item,current_user,datapoint):
    get_genre = item_data[item_data['movie id']==current_item]
    col = ("unknown" , "Action" , "Adventure",  
                     "Animation" ,"Children\'s" , "Comedy" , "Crime" ,
                     "Documentary" ,"Drama" , "Fantasy" ,"Film-Noir" ,
                     "Horror" , "Musical" ,"Mystery" , "Romance" ,
                     "Sci-Fi" ,"Thriller" ,"War" , "Western")
    genre_list = []
    for each in col:
        if get_genre[each].any()==True :
            genre_list.append(each)
    
    movie_list = []
    for i in range(len(genre_list)):
        mylist = list(item_data[item_data[genre_list[i]]==1]['movie id'])
        movie_list.append(mylist)
    final_list= []  
    for lis in movie_list:
        for every in lis:
            final_list.append(every)
    newarr=[]
    current_user_movies = list(datapoint.index)
    for each in current_user_movies:
        if each in final_list:
            newarr.append(each)
    if len(newarr)==0:
        return 0.0
    rating_given=0
    for each in newarr:
        rating_given+=ratings_df.iloc[current_user-1][each] 
    avg_rating = (round(rating_given/len(newarr)))
    return avg_rating

#item_data.columns

In [ ]:
#del u_data['timestamp']
ratings_df = u_data.pivot_table(values='rating',index='user id',columns = 'item id')
ratings_df.fillna(value=0,inplace=True,axis=1)


ratings_df1 = ratings_df.copy()
ratings_df1['user id'] = ratings_df1.index

ratings_df1 = ratings_df1.merge(user_data,left_on='user id',right_on='user id',how='inner')
ratings_df1.head()
ratings_df.head()
oc1 = ['educator','engineer' ,'healthcare','lawyer','programmer','scientist','writer','doctor']
oc2 = ['artist','Administrator','executive','marketing','entertainment']
oc3 = ['librarian','salesman','technician']
oc4 = ['homemaker','none','other','retired']
for ix,row in ratings_df1.iterrows():
    if row['occupation'] in oc1:
        ratings_df1.set_value(ix,'occupation',1)
    elif row['occupation'] in oc2:
        ratings_df1.set_value(ix,'occupation',2)
    elif row['occupation'] in oc2:
        ratings_df1.set_value(ix,'occupation',3)
    else:
        ratings_df1.set_value(ix,'occupation',4)
ratings_df1.index = user_data['user id']
ratings_df1.drop(['user id','zip code'],inplace = True, axis = 1)
ratings_df1.head()

info = ratings_df1.iloc[:,-3:]


In [ ]:
rating=[]
count = 0
for index, rows in u1_test.iterrows():
    current_item = rows['item id']
    current_user = rows['user id'] 
    count+=1
   # print(count)
    if len(u_data[u_data['item id']==current_item]['rating'])!=0:
        rating.append(round(u_data[u_data['item id']==current_item]['rating'].mean()))
    else:
        rating.append(0)


In [ ]:
print("Naive method:",MAD(rating))

In [ ]:
k = 100
count = 1
rating1 = []
rating2 = []
rating3 = []
for index, rows in u1_test.iterrows():
    current_item = rows['item id']
    current_user = rows['user id'] 

    #Drop items that haven't rated for the movie because they are useless in prediction
    if current_item not in ratings_df.columns:
        rating1.append(3)
        rating2.append(3)
        rating3.append(3)
        continue
    current_train = ratings_df[ratings_df[current_item]!=0.0]

    datapoint = ratings_df.iloc[current_user-1].drop([current_item])
    datapoint1 = datapoint.iloc[datapoint.nonzero()[0]]
    
    current_train = current_train[list(datapoint1.index)]
    
    train_item = current_train  
    #print(train_item)
    
    train_item = np.array(train_item)
    datapoint1= np.array(datapoint1)
    diff = abs(train_item-datapoint1)
    euc = np.sum(diff,axis = 1)

    close_users1 = []
    close_users2 = []
    close_users3 = []
    
    mylist1 = list(euc.argsort()[0:10])
    mylist2 = list(euc.argsort()[0:50])
    mylist3 = list(euc.argsort()[0:k])
    
    counter1 = 0
    for each in current_train.index:
        if counter1 in mylist1:
            close_users1.append(each)
        counter1+=1
    #print(close_users1)
    
    counter2 = 0
    for each in current_train.index:
        if counter2 in mylist2:
            close_users2.append(each)
        counter2+=1
    #print(close_users2)
    
    counter3 = 0
    for each in current_train.index:
        if counter3 in mylist3:
            close_users3.append(each)
        counter3+=1
    #print(close_users3)
   
    total1 = 0
    total2 = 0
    total3 = 0
    k_count1 = 0
    k_count2 = 0
    k_count3 = 0
    
    for every in close_users1:

            total1+=ratings_df.iloc[every-1][current_item]
            k_count1+=1
            
    for every in close_users2:

            total2+=ratings_df.iloc[every-1][current_item]
            k_count2+=1
            
    for every in close_users3:
            total3+=ratings_df.iloc[every-1][current_item]
            k_count3+=1
            
    if k_count1==0:
        rating1.append(3.0)
        rating2.append(3.0)
        rating3.append(3.0)
        print(count)
        count+=1
        continue
        
    rating1.append(round(total1/k_count1))
    rating2.append(round(total2/k_count2))
    rating3.append(round(total3/k_count3))

    $print(count)
    count+=1



In [ ]:
print("Improved method(k-10)",MAD(rating1))
print("Improved method(k-50)",MAD(rating2))
print("Improved method(k-100)",MAD(rating3))

In [ ]:
k = 100
count = 1
rating1 = []
rating2 = []
rating3 = []
for index, rows in u1_test.iterrows():
    current_item = rows['item id']
    current_user = rows['user id'] 

    #Drop items that haven't rated for the movie because they are useless in prediction
    if current_item not in ratings_df.columns:
        rating1.append(3)
        rating2.append(3)
        rating3.append(3)
        continue
        
    current_train = ratings_df[ratings_df[current_item]!=0.0]

    datapoint = ratings_df.iloc[current_user-1].drop([current_item])
    #datapoint.drop(['user id','age','zip code','occupation','gender'],inplace = True)
    datapoint1 = datapoint.iloc[datapoint.nonzero()[0]]
    current_train = current_train[list(datapoint1.index)]
    train_item = current_train  
    #print(train_item)
    
    user_info = current_train.index
    
    user = info.iloc[user_info-1]
    user_now = info.iloc[current_user-1]

    train_item = np.array(train_item)
    user = np.array(user)
    datapoint2 = datapoint1
    
    datapoint1= np.array(datapoint1)
    user_now = np.array(user_now)
    #print(user)
    #print(user_now)
    
    
    diff = abs(train_item-datapoint1)
    diff_user = abs(user-user_now)
    #print(diff_user)
    
    
    #euc = pow(np.sum(pow(diff,2),axis = 1),0.5)
    #euc1 = pow(np.sum(pow(diff_user,2),axis = 1),0.5)
    euc= np.sum(diff,axis = 1)
    euc1 = np.sum(diff_user,axis = 1)
    
    euc1 = euc1
    euc = euc
    
    euc = euc1 + euc
    
    close_users1 = []
    close_users2 = []
    close_users3 = []
    
    mylist1 = list(euc.argsort()[0:10])
    mylist2 = list(euc.argsort()[0:50])
    mylist3 = list(euc.argsort()[0:k])

    counter1 = 0
    for each in current_train.index:
        if counter1 in mylist1:
            close_users1.append(each)
        counter1+=1
    #print(close_users1)
    
    counter2 = 0
    for each in current_train.index:
        if counter2 in mylist2:
            close_users2.append(each)
        counter2+=1
    #print(close_users2)
    
    counter3 = 0
    for each in current_train.index:
        if counter3 in mylist3:
            close_users3.append(each)
        counter3+=1
    #print(close_users3)
   
    total1 = 0
    total2 = 0
    total3 = 0
    k_count1 = 0
    k_count2 = 0
    k_count3 = 0
    
    for every in close_users1:

            total1+=ratings_df.iloc[every-1][current_item]
            k_count1+=1
            
    for every in close_users2:

            total2+=ratings_df.iloc[every-1][current_item]
            k_count2+=1
            
    for every in close_users3:
            total3+=ratings_df.iloc[every-1][current_item]
            k_count3+=1
            
    if k_count1==0:
        rating1.append(3.0)
        rating2.append(3.0)
        rating3.append(3.0)
        print(count)
        print("No user")
        count+=1
        continue
        
    myavg = myratings(current_item,current_user,datapoint2)
    
    val1 = (0.8*round(total1/k_count1) + 0.2*myavg)
    val2 = (0.8*round(total2/k_count2) + 0.2*myavg)
    val3 = (0.8*round(total3/k_count3) + 0.2*myavg)
    """
    val1 = (round(total1/k_count1) + myavg)/2
    val2 = (round(total2/k_count2) + myavg)/2
    val3 = (round(total3/k_count3) + myavg)/2
    """
    rating1.append(val1)
    rating2.append(val2)
    rating3.append(val3)

    print(count)
    count+=1



In [ ]:
print("Improved method(k-10) with age,gender,occupation",MAD(rating1))
print("Improved method(k-50) with age,gender,occupation",MAD(rating2))
print("Improved method(k-100) with age,gender,occupation",MAD(rating3))